***Before doing anything, close Zotero!***

In [1]:
import time
import sqlite3
import sys

First, just copy the `sqlite` file to this directory for manipulation.

In [2]:
%cp /Users/boyle/Library/Application\ Support/Firefox/Profiles/l1i89wc9.default/zotero/zotero.sqlite zotero.sqlite
%cp /Users/boyle/Library/Application\ Support/Firefox/Profiles/l1i89wc9.default/zotero/zotero.sqlite zotero.sqlite.bak

Now, connect to it, and give us a cursor in the file.

In [3]:
con = sqlite3.connect('zotero.sqlite')
cur = con.cursor() 

The following gives the list of tables in this database file:

In [4]:
cur.execute("select name from sqlite_master where type = 'table';").fetchall()

[('version',),
 ('settings',),
 ('itemDataValues',),
 ('itemData',),
 ('itemNotes',),
 ('itemAttachments',),
 ('itemTags',),
 ('itemSeeAlso',),
 ('creatorData',),
 ('itemCreators',),
 ('collectionItems',),
 ('savedSearchConditions',),
 ('fulltextItems',),
 ('fulltextWords',),
 ('fulltextItemWords',),
 ('annotations',),
 ('highlights',),
 ('proxies',),
 ('proxyHosts',),
 ('zoteroDummyTable',),
 ('deletedItems',),
 ('collections',),
 ('creators',),
 ('items',),
 ('savedSearches',),
 ('tags',),
 ('libraries',),
 ('users',),
 ('groups',),
 ('groupItems',),
 ('storageDeleteLog',),
 ('relations',),
 ('syncDeleteLog',),
 ('customItemTypes',),
 ('customFields',),
 ('customItemTypeFields',),
 ('customBaseFieldMappings',),
 ('itemTypes',),
 ('itemTypesCombined',),
 ('fieldFormats',),
 ('fields',),
 ('fieldsCombined',),
 ('itemTypeFields',),
 ('itemTypeFieldsCombined',),
 ('baseFieldMappings',),
 ('baseFieldMappingsCombined',),
 ('charsets',),
 ('fileTypes',),
 ('fileTypeMimeTypes',),
 ('creatorT

Let's take a look at the layout and contents of some of these tables.

In [5]:
print(cur.execute("select sql from sqlite_master where name = 'collections';").fetchall()[0][0])
print(cur.execute("select * from collections;").fetchall())

CREATE TABLE collections (
    collectionID INTEGER PRIMARY KEY,
    collectionName TEXT NOT NULL,
    parentCollectionID INT DEFAULT NULL,
    dateAdded TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    dateModified TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    clientDateModified TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    libraryID INT,
    key TEXT NOT NULL,
    UNIQUE (libraryID, key),
    FOREIGN KEY (parentCollectionID) REFERENCES collections(collectionID)
)
[(1, 'Quantum Foam and Causal Sets', 8, '2009-02-04 22:01:28', '2014-02-18 15:09:48', '2014-02-18 15:09:48', None, '883NBR42'), (2, 'NR/DA', 8, '2009-02-04 22:01:37', '2015-05-27 21:23:45', '2015-05-27 21:23:45', None, '6C6HU2R5'), (3, 'Gravitational Waves', 8, '2009-02-15 20:03:24', '2015-05-27 21:20:23', '2015-05-27 21:20:23', None, '96M57BK6'), (4, 'Black Holes', 8, '2009-02-04 22:01:32', '2014-01-03 16:32:40', '2014-01-03 16:32:40', None, 'I79TN2G5'), (5, 'Phenom', 34, '2010-12-21 16:19:15', '2011-03-19 03:08:58

So the `collections` table just gives me all the collections I see in the left-hand section of zotero, as expected.

In [6]:
print(cur.execute("select sql from sqlite_master where name = 'itemData';").fetchall()[0][0])
print(cur.execute("select * from itemData;").fetchall())

CREATE TABLE itemData (
    itemID INT,
    fieldID INT,
    valueID,
    PRIMARY KEY (itemID, fieldID),
    FOREIGN KEY (itemID) REFERENCES items(itemID),
    FOREIGN KEY (fieldID) REFERENCES fields(fieldID),
    FOREIGN KEY (valueID) REFERENCES itemDataValues(valueID)
)
[(1, 110, 1), (1, 14, 2), (1, 1, 3), (1, 12, 4), (1, 90, 5), (1, 62, 6), (1, 27, 7), (2, 110, 8), (2, 1, 9), (2, 27, 10), (3, 110, 11), (3, 14, 12), (3, 1, 13), (3, 12, 14), (3, 90, 15), (3, 62, 6), (3, 27, 16), (4, 110, 17), (4, 1, 18), (4, 27, 19), (7, 110, 27), (7, 14, 28), (7, 1, 29), (7, 12, 30), (7, 90, 31), (7, 62, 6), (7, 27, 32), (7, 116, 33), (8, 110, 34), (8, 1, 35), (8, 27, 36), (9, 110, 37), (9, 14, 28), (9, 1, 38), (9, 12, 39), (9, 90, 40), (9, 62, 6), (9, 27, 41), (9, 116, 42), (10, 110, 43), (10, 1, 44), (10, 27, 41), (13, 110, 46), (13, 14, 28), (13, 1, 47), (13, 22, 48), (13, 12, 49), (13, 90, 50), (13, 62, 6), (13, 27, 51), (14, 110, 52), (14, 1, 53), (14, 27, 54), (19, 110, 63), (19, 14, 64), (19, 

The 'itemData' table is funny.  It's just a collection of integers, telling me

  1. which item each piece of data is associated with
  2. what kind of data is stored in this piece of data
  3. which `valueID` stores the data
  
So evidently, `valueID` points to another table `itemDataValues` which stores each of the actual pieces of data, and `fieldID` points to *another* table `fields`, which stores the type of data corresponding to that integer.

First, let's see what these `fieldID`s actually mean:

In [7]:
print(cur.execute("select sql from sqlite_master where name = 'fields';").fetchall()[0][0])
print(cur.execute("select * from fields;").fetchall())

CREATE TABLE fields (
    fieldID INTEGER PRIMARY KEY,
    fieldName TEXT,
    fieldFormatID INT,
    FOREIGN KEY (fieldFormatID) REFERENCES fieldFormats(fieldFormatID)
)
[(1, 'url', None), (2, 'rights', None), (3, 'series', None), (4, 'volume', None), (5, 'issue', None), (6, 'edition', None), (7, 'place', None), (8, 'publisher', None), (10, 'pages', None), (11, 'ISBN', None), (12, 'publicationTitle', None), (13, 'ISSN', None), (14, 'date', None), (15, 'section', None), (18, 'callNumber', None), (19, 'archiveLocation', None), (21, 'distributor', None), (22, 'extra', None), (25, 'journalAbbreviation', None), (26, 'DOI', None), (27, 'accessDate', None), (28, 'seriesTitle', None), (29, 'seriesText', None), (30, 'seriesNumber', None), (31, 'institution', None), (32, 'reportType', None), (36, 'code', None), (40, 'session', None), (41, 'legislativeBody', None), (42, 'history', None), (43, 'reporter', None), (44, 'court', None), (45, 'numberOfVolumes', None), (46, 'committee', None), (48, 'as

I'm interested in URLs, which have `fieldID` 1.

Now, I want to find which pieces of data are stored at those `valueID`s:

In [8]:
print(cur.execute("select sql from sqlite_master where name = 'itemDataValues';").fetchall()[0][0])
print(cur.execute("select * from itemDataValues;").fetchall())

CREATE TABLE itemDataValues (
    valueID INTEGER PRIMARY KEY,
    value UNIQUE
)
[(1, 'What if Time Really Exists?'), (2, '2008-11-23 2008-11-23'), (3, 'http://arxiv.org/abs/0811.3772'), (4, '0811.3772'), (5, 'Despite the obvious utility of the concept, it has often been argued that time does not exist. I take the opposite perspective: let\'s imagine that time does exist, and the universe is described by a quantum state obeying ordinary time-dependent quantum mechanics. Reconciling this simple picture with the known facts about our universe turns out to be a non-trivial task, but by taking it seriously we can infer deep facts about the fundamental nature of reality. The arrow of time finds a plausible explanation in a "Heraclitean universe," described by a quantum state eternally evolving in an infinite-dimensional Hilbert space.'), (6, 'arXiv.org'), (7, '2009-02-04 21:33:03'), (8, '0811.3772 PDF'), (9, 'http://www.arxiv.org/pdf/0811.3772.pdf'), (10, '2009-02-04 21:33:04'), (11, 'Time

I can run a query to select the `value`s corresponding to URLs, and only show those URLs containing `'proxy'`:

In [9]:
[url[0]
 for url in cur.execute("SELECT value FROM itemDataValues WHERE valueID IN (SELECT valueID FROM itemData where fieldID=1)").fetchall()
 if 'proxy' in url[0]]

['http://link.aps.org.clsproxy.library.caltech.edu/abstract/PRD/v78/e124020',
 'http://scitation.aip.org.clsproxy.library.caltech.edu/getpdf/servlet/GetPDFServlet?filetype=pdf&id=PRVDAQ000078000012124020000001&idtype=cvips&prog=normal',
 'http://link.aps.org.clsproxy.library.caltech.edu/abstract/PRD/v78/e044039',
 'http://onlinelibrary.wiley.com.proxy.library.cornell.edu/doi/10.1111/j.1365-2966.2009.15427.x/abstract;jsessionid=BF1B9266CFD6F052747D05BB4714066C.d03t04',
 'http://scitation.aip.org.clsproxy.library.caltech.edu/getpdf/servlet/GetPDFServlet?filetype=pdf&id=PRVDAQ000078000004044039000001&idtype=cvips&prog=normal',
 'http://scitation.aip.org.proxy.library.cornell.edu/content/aip/proceeding/aipcp/10.1063/1.4825543',
 'http://link.aps.org.clsproxy.library.caltech.edu/abstract/PRD/v79/e084011',
 'http://scitation.aip.org.clsproxy.library.caltech.edu/getpdf/servlet/GetPDFServlet?filetype=pdf&id=PRVDAQ000079000008084011000001&idtype=cvips&prog=normal',
 'http://www.iop.org.clsproxy

In [10]:
len(_)

434

Here's another way to do the same thing, doing that string match just within SQL:

In [11]:
[url[0]
 for url in cur.execute("SELECT value FROM itemDataValues "
                        "WHERE valueID IN (SELECT valueID FROM itemData where fieldID=1) and value LIKE '%proxy%'").fetchall()]

['http://link.aps.org.clsproxy.library.caltech.edu/abstract/PRD/v78/e124020',
 'http://scitation.aip.org.clsproxy.library.caltech.edu/getpdf/servlet/GetPDFServlet?filetype=pdf&id=PRVDAQ000078000012124020000001&idtype=cvips&prog=normal',
 'http://link.aps.org.clsproxy.library.caltech.edu/abstract/PRD/v78/e044039',
 'http://onlinelibrary.wiley.com.proxy.library.cornell.edu/doi/10.1111/j.1365-2966.2009.15427.x/abstract;jsessionid=BF1B9266CFD6F052747D05BB4714066C.d03t04',
 'http://scitation.aip.org.clsproxy.library.caltech.edu/getpdf/servlet/GetPDFServlet?filetype=pdf&id=PRVDAQ000078000004044039000001&idtype=cvips&prog=normal',
 'http://scitation.aip.org.proxy.library.cornell.edu/content/aip/proceeding/aipcp/10.1063/1.4825543',
 'http://link.aps.org.clsproxy.library.caltech.edu/abstract/PRD/v79/e084011',
 'http://scitation.aip.org.clsproxy.library.caltech.edu/getpdf/servlet/GetPDFServlet?filetype=pdf&id=PRVDAQ000079000008084011000001&idtype=cvips&prog=normal',
 'http://www.iop.org.clsproxy

In [12]:
len(_)

434

Now I'll try to update the table to replace those strings:

In [13]:
cur.execute("UPDATE itemDataValues SET value = replace( replace( value, '.proxy.library.cornell.edu', '' ), '.clsproxy.library.caltech.edu', '')"
            "WHERE (valueID IN (SELECT valueID FROM itemData where fieldID=1) and value LIKE '%proxy%')")

IntegrityError: UNIQUE constraint failed: itemDataValues.value

The `itemDataValues` table did indeed insist that the `value` field should be unique.  But I don't really understand why there should be clashes.  Maybe I got the same paper at Caltech and Cornell, via the respective proxies.  I'll try going through them individually.  (This is presumably slower than pure SQL, but there shouldn't be so many that it'll be a problem.)

In [14]:
itemDataValues = cur.execute("SELECT * FROM itemDataValues "
                             "WHERE valueID IN (SELECT valueID FROM itemData where fieldID=1) and value LIKE '%proxy%'").fetchall()
for valueID, value in itemDataValues:
    new_value = value.replace('.proxy.library.cornell.edu', '' ).replace('.clsproxy.library.caltech.edu', '')
    #print(valueID, value, '\n    ', new_value); sys.stdout.flush()
    try:
        cur.execute("UPDATE itemDataValues SET value = '{0}'".format(new_value)
                    + "WHERE valueID = {0}".format(valueID))
    except sqlite3.IntegrityError:
        print(cur.execute("SELECT * from itemDataValues WHERE value LIKE '%{0}%' OR value LIKE '%{1}%'".format(value, new_value)).fetchall())
        print("")

[(1347, 'http://prd.aps.org.proxy.library.cornell.edu/pdf/PRD/v49/i6/p2658_1'), (2448, 'http://prd.aps.org/pdf/PRD/v49/i6/p2658_1')]

[(3349, 'http://prd.aps.org.proxy.library.cornell.edu/pdf/PRD/v58/i12/e124031'), (4002, 'http://prd.aps.org/pdf/PRD/v58/i12/e124031')]

[(4800, 'http://prd.aps.org/pdf/PRD/v82/i4/e044025'), (4803, 'http://prd.aps.org.proxy.library.cornell.edu/pdf/PRD/v82/i4/e044025')]

[(8824, 'http://www.sciencedirect.com/science/article/pii/0040938364900035/pdf?md5=f55e052338189ca79550217063b0aedf&pid=1-s2.0-0040938364900035-main.pdf'), (10341, 'http://www.sciencedirect.com.proxy.library.cornell.edu/science/article/pii/0040938364900035/pdf?md5=f55e052338189ca79550217063b0aedf&pid=1-s2.0-0040938364900035-main.pdf')]



These all appear to be double-PDFs, presumably from Duplicate Items that I've merged.  I'll just go through and delete them all, and run this notebook again...

In [15]:
if con:
    con.commit()
    con.close()

Now, I'm too scared to actually add this as an executable statement to this notebook, but the following command will also need to be executed

    cp zotero.sqlite /Users/boyle/Library/Application\ Support/Firefox/Profiles/l1i89wc9.default/zotero/zotero.sqlite